# The fundarmental group of $\Sigma_{g,1}$ 

In [1]:
require './freegroup.rb'

true

## symplectic generators

In [2]:
genus = 3
alphabet = %w(a b c d e f)

Gens = [Letter.new] + alphabet.map{|x| Letter.new(x)}
Gens.freeze
p Gens[1..-1].map(&:show)

a = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:first)
b = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:last)
p a.map(&:show)
nil

["a", "b", "c", "d", "e", "f"]
["1", "a", "c", "e"]


In [3]:
comms = []
(Gens.size/2 + 1).times do |i|
  comms << Group.commutator(a[i], b[i])
end
p comms.map(&:show)

["1111", "abAB", "cdCD", "efEF"]


["1111", "abAB", "cdCD", "efEF"]

In [4]:
Rseq = []
(Gens.size/2).times do |i|
  Rseq += [a[i+1], b[i+1].inverse, a[i+1].inverse, b[i+1]]
end
Rseq.freeze
p Rseq.map(&:show) #.index('A')
nil

<main>:2: warning: already initialized constant Rseq
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here


["a", "B", "A", "b", "c", "D", "C", "d", "e", "F", "E", "f"]


## Random Word generator

In [5]:
def word_generator(length=5)
  rstr = ''
  length.times{|k| rstr += Rseq.sample.to_s}
  return Word.new(rstr)
end
def wordgen(length=5)
  word_generator(length)
end;nil

In [6]:
p wordgen(10).show

"EebCDeaCfE"


"EebCDeaCfE"

# Goldman-Turaev bialgebra

## Linking of a pair of divisions

In [7]:
def ep(h1, h2)
  result = h1[:sign]*h2[:sign]
  result *= if h1[:letter].show == h2[:letter].show
    (h1[:index] < h2[:index]) ? h2[:sign] : h1[:sign]*(-1)
  else
    (Rseq.index(h1[:letter]) < Rseq.index(h2[:letter])) ? 1 : -1
  end
  return result
end
#--------------------------------------------

def lk(h1, h2) # h1={word: w1, div: div1}, h2={word: w2, div: div2})
  div_hashs = [h1,h2].map do |h|
    w = h[:word].flatten
    k = h[:div]
    s = w.size
    [{index: (k%s)+1, letter: w[k%s], sign: 1}, {index: k, letter: w[k-1].inverse, sign: -1}]
  end
  sum = 0
  2.times do |i|
    2.times{|j| sum += ep(div_hashs[0][i], div_hashs[1][j])}
  end
  return (-1)*(sum/2)
end; nil

## Expansion Class

In [8]:
class Expansion < Array
  def show(simplify_level=0)
    expn = case simplify_level
    when 1
      self.delete_if{|h| h[:coeff] == 0}
    when 2
      self.simplify
    else 
      self.reverse
    end
    #---
    mstr = expn.map do |h|
      coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
      words = h[:words]
      term = (words.size == 1) ? words[0].show : words.map(&:cyclic_reduce).join("\u{2227}")
      coeff + term
    end.join(' + ')
    return (mstr.empty?) ? "0" : mstr
  end

  def simplify()
    #-----
    expn = self.class.new
    monomials = self.each_with_object([]) do |mono, arr|
      if mono[:coeff] != 0        
        mono_reduced = Hash(divs: mono[:divs], coeff: mono[:coeff], words: mono[:words].map(&:cyclic_reduce))
        arr << mono_reduced
      end
    end
    while monomials.size > 0
      pop = monomials.pop
      pop[:divs] = [pop[:divs]]
      #---
      monomials.each do |mono|
        switch = same_term?(mono[:words], pop[:words])
        unless switch.nil?
          pop[:coeff] += mono[:coeff] * switch
          pop[:divs] << mono[:divs]
          mono[:coeff] = nil
        end
      end
      #---
      expn << pop if pop[:coeff] != 0
      #---
      monomials.delete_if{|mono| mono[:coeff].nil?}
    end
    #-----
    if expn.empty?
      return (self.class.new << {divs: [], coeff: 0, words: [Word.new]})
    else
      return expn
    end
  end
  
  private
  def same_term?(ws1, ws2)
    ws1.size.times do |k|
      flags = ws1.zip(ws2.rotate(k)).map{|couple| couple[0].conjugate?(couple[1])}
      if flags.all?{|tf| tf}
        return (-1)**k
        break
      end
    end
    return nil
  end
end; nil

In [77]:
10000.times do |k|
  expn = Expansion.new() 
  5.times do |i|
    cf = ( (-1)**(i.modulo(3)) - (-1)**((i+1).modulo(3)) )/2
    ws = [2,3].map{|k| word_generator(k)}
    expn << {divs: '', coeff: cf, words: ws}
  end
  if expn.simplify.size < 4
    3.times{|i| p "simplify_level #{i}: " + expn.show(i)}
    break
  end
end; nil

"simplify_level 0: (-1)CB∧cDa + cb∧aBa + (0)CF∧AbF + (-1)BC∧Dac + FA∧fbd"
"simplify_level 1: FA∧fbd + (-1)BC∧Dac + cb∧aBa + (-1)CB∧cDa"
"simplify_level 2: (-2)CB∧cDa + cb∧aBa + FA∧fbd"


## Goldman bracket

In [10]:
def bracket(w1, w2)
  """
  retern an Expansion (an Array of Hashes {divs: *, coeff: *, words: *})
  """
  #-----
  polynomial = Expansion.new
  if w1.show != '1' and w2.show != '1' 
    w1.size.times do |i|
      #--- divide two words each ---
      div1 = i+1
      nw1 = w1.cyclic_permutation(div1)
      w2.size.times do |j|
        div2 = j+1
        #--- linking of the pair of divisions ---
        lk = lk({word: w1, div: div1}, {word: w2, div: div2})
        nw2 = w2.cyclic_permutation(div2)
        #---
        polynomial << {divs: [div1, div2], coeff: lk, words: [nw1*nw2]}
      end
    end
  end
  return polynomial
end; nil

In [11]:
# w1 = Word.new(a[1], a[2], b[1].inverse)
# w2 = Word.new(b[2], a[1], a[1].inverse) #, b[2])
w1, w2 = wordgen(2), wordgen(3)
printf [w1, w2].map(&:show).join(', ')  + "\n-------\n"

bra = bracket(w1, w2)

# bra.each do |h|
#   h.each{|k,v| p "#{k}: " + ((k == :words) ? v[0].show : v.to_s)}
#   printf "---\n"
# end

p bra.show
p bra.simplify.show
; nil

Ac, cCF
-------
"(-1)Ac.cCF + Ac.FcC + (0)Ac.CFc + cA.cCF + (-1)cA.FcC + (0)cA.CFc"
"(0)1"


## Turaev cobraket

In [12]:
def cobracket(myw, verbose: false)
  """
  return an Expansion (an Array of Hashes {divs: *, coeff: *, words: *})
  """
  polynomial = Expansion.new
  unless myw.show == '1' 
    divs_combinations = (1..myw.size).to_a.combination(2)  #2つの div の組み合わせの全体
    divs_combinations.each do |combi|
      #--- linking of the pair of divisions ---
      lk = lk({word: myw, div: combi[0]},{word: myw, div: combi[1]})
      #--- divide the word --- 
      facs = myw.flatten.factors
      words = Array.new(2)
      #--- [[TODO]] 以下の2つの Word の定義に cyclic_permutation を使えないか?
      words[0] = Word.new(facs[0..combi[0]-1]+facs[combi[1]..-1])
      words[1] = Word.new(facs[combi[0]..combi[1]-1])    
      polynomial << {divs: combi, coeff: lk, words: words}
    end
  end
  #-----
  return polynomial
end; nil

In [13]:
#mw = Word.new(b[1], a[1], b[1]) #a1*Group.commutator(b[1], a[2]) #comms[1] #
mw = wordgen(5)
printf mw.show + "\n-------\n"

cobra = cobracket(mw, verbose: false) #true) #

# cobra.each do |h|
#   h.each do |k,v| 
#     p "#{k}: " + ((k == :words) ? v[0].show : v.to_s)
#   end
#   printf "---\n"
# end

p cobra.show
p cobra.simplify.show
nil

EdAdC
-------
"(0)EdAd∧C + (-1)EdA∧dC + (0)EdAC∧d + Ed∧AdC + EdC∧Ad + (0)EddC∧A + (0)E∧dAdC + (0)EC∧dAd + (-1)EdC∧dA + (0)EAdC∧d"
"Ed∧AdC + (-1)EdA∧dC"


## Execution

In [71]:
a1, b1 = Word.new(a[1]), Word.new(b[1])
c = (a1*Word.new(b[1])*(a1.inverse)).flatten

samples = {
  s0: [
    Word.new,
    a1, 
    a1*Word.new(b[2]),
    a1*(Word.new(b[2]).inverse),
    a1*c*(c.inverse),
  ],
  s1: [
    a1^4,
    c,
    comms[1],
    Word.new(b[1])*a1*Word.new(b[1])
  ],
  s2: [
    a1*(comms[2].inverse),
    a1*(c.inverse)*(comms[2].inverse)*c,
    a1*(comms[2].inverse)*Group.commutator(comms[2], c.inverse),
    (a1*(comms[2].inverse)*Group.commutator(comms[2], c.inverse)).cyclic_reduce
  ],
  s3: [
    a1*comms[2],
    a1*(comms[2]^2),
    a1*comms[2]*c*comms[2]*(c.inverse),
    a1*(comms[2]^2)*Group.commutator(comms[2].inverse,c)
  ],
  s4: [
    a1*(Word.new(a[2])^3),
    a1*(Word.new(a[2], b[2].inverse)^3)
  ],
  s5: [
    a1*Group.commutator(a[2], a[3]),
    a1*(comms[2]*comms[3])*Group.commutator(a[2], a[3])*((comms[2]*comms[3]).inverse)
  ],
  s6: [
    a1*Group.commutator(b[1], a[2]),
    a1*(a1.inverse)*Word.new(b[1])*a1,
    c*(a1.inverse)*a1,
    c*(c.inverse),
    Word.new(a[1], b[1], b[1].inverse, b[2])
  ],
  s7: [
    a1*(Group.conjugate(a1, b1.inverse))*a1*(b1.inverse),
    a1*b1*((a1*(b1.inverse))^2),
    a1*Group.conjugate(b1.inverse, a1)*a1*(b1.inverse),
    (a1*Group.conjugate(b1.inverse, a1)*a1*(b1.inverse)).contract,
    a1*a1*(Word.new(b[2].inverse)^2)
    ]
  }

arr = samples #.slice(:s7) #:s0, :s6) #:s2) #, :s3) #:s0, :s1) #
arr.each do |k, v|
  printf "--- #{k} ---\n "
  v.each{|s| puts s.show + "  |--\u{03B4}-->  " + cobracket(s).show(2) }
end;nil

--- s0 ---
 1  |--δ-->  (0)1
a  |--δ-->  (0)1
a.d  |--δ-->  (-1)a∧d
a.D  |--δ-->  (0)1
a.abA.aBA  |--δ-->  1∧a
--- s1 ---
 a.a.a.a  |--δ-->  aa∧aa
abA  |--δ-->  1∧b
abAB  |--δ-->  (0)1
b.a.b  |--δ-->  (0)1
--- s2 ---
 a.dcDC  |--δ-->  (-1)a∧dcDC
a.aBA.dcDC.abA  |--δ-->  1∧aBAdcDCab
a.dcDC.cdCDaBAdcDCabA  |--δ-->  (2)1∧aBAdcDCab
aBAdcDCab  |--δ-->  (0)1
--- s3 ---
 a.cdCD  |--δ-->  (0)1
a.cdCD.cdCD  |--δ-->  acdCD∧cdCD
a.cdCD.abA.cdCD.aBA  |--δ-->  (0)1
a.cdCD.cdCD.dcDCabAcdCDaBA  |--δ-->  (0)1
--- s4 ---
 a.c.c.c  |--δ-->  acc∧c + ac∧cc
a.cD.cD.cD  |--δ-->  acDcD∧cD + acD∧cDcD
--- s5 ---
 a.ceCE  |--δ-->  (-1)aceC∧E + ace∧CE + (-1)ac∧C
a.cdCD.efEF.ceCE.feFEdcDC  |--δ-->  (-1)acdCDefEFceCfeFEdcDC∧E + acdCDefEFcefeFEdcDC∧CE + (-1)acdCDefEFcfeFEdcDC∧C
--- s6 ---
 a.bcBC  |--δ-->  (-1)abcB∧C + abc∧BC + (-1)abC∧cB
a.A.b.a  |--δ-->  (0)1
abA.A.a  |--δ-->  1∧b
abA.aBA  |--δ-->  (0)1
abBd  |--δ-->  (-1)a∧d + (-1)ad∧1
--- s7 ---
 a.Bab.a.B  |--δ-->  (0)1
a.b.a.B.a.B  |--δ-->  (0)1
a.aBA.a.B  |-

# Conditions

## Jacobi Identity

$$\nabla\circ(\nabla\otimes 1)\circ N = 0$$

In [15]:
three_words = Array.new(3)
max_length = 2

three_words.map!{|w| word_generator(rand(1..max_length))}
puts three_words.map(&:show).join(" \u{2297} ") + "  |---> " + "\n------"

total = Expansion.new
3.times do |k|
  bra = bracket(three_words[0],three_words[1])
  puts "\t\u{2207}(" + three_words[0..1].map(&:show).join(" \u{2297} ") + ") \u{2297} " + three_words[2].show + " ="
  bra.each do |h|
    bra2 = bracket(h[:words][0], three_words[2])
    puts "\t\t+ (#{bra2.show(1)}) \u{2297} #{three_words[2].show}"
    bra2.each do |hh|
      hh[:coeff] *= h[:coeff]
      total << hh
    end
  end
  three_words.rotate!
end

printf "---------\n"
p total.show(1)
printf "---------\n"
p total.show(2)
nil

C ⊗ ae ⊗ D  |---> 
------
	∇(C ⊗ ae) ⊗ D =
		+ (Cea.D) ⊗ D
		+ (Cae.D) ⊗ D
	∇(ae ⊗ D) ⊗ C =
		+ ((-1)Dea.C) ⊗ C
		+ ((-1)Dae.C) ⊗ C
	∇(D ⊗ C) ⊗ ae =
		+ (0) ⊗ ae
---------
"0"
---------
"(0)1"


In [16]:
three_words = Array.new(3)
max_length = 3
count = 500

count.times do |k|
  three_words.map!{|w| word_generator(rand(1..max_length))}
  total = Expansion.new
  3.times do |k|
    bra = bracket(three_words[0],three_words[1])
#     puts "\u{2207}(" + three_words[0..1].map(&:show).join(" \u{2297} ") + ") \u{2297} " + three_words[2].show + " ="
    bra.each do |h|
      bra2 = bracket(h[:words][0], three_words[2])
#       puts "+ (#{bra2.show}) \u{2297} #{three_words[2].show}"
      bra2.each do |hh|
        hh[:coeff] *= h[:coeff]
        total << hh
      end
    end
    three_words.rotate!
  end
  result = total.show(2)
  puts three_words.map(&:show).join(" \u{2297} ") + "  |--> " + result if result != '(0)1'
end; nil

## Involutivity for $\nabla$ and $\delta$

$$\nabla\circ \delta = 0$$

In [17]:
myw = word_generator(6)
#myw = Word.new('ffA')
puts myw.show + "\n======"

cobra = cobracket(myw)
puts "\u{0394}(#{myw.show}) = " + cobra.show(1)
puts "---"

total = Expansion.new
cobra.each do |h|
  coeff = h[:coeff]
  bra = bracket(*h[:words])
  bra.each do |mono|
    mono[:coeff] *= coeff
    total << mono
  end
  #--- display ---
  tensor = h[:words].map(&:show).join(" \u{2297} ")
  puts "\t\u{2207}(#{tensor}) = #{bra.show(1)}"
  #---------------
end

puts "======\n" + total.show(1) + "  =  " + total.show(2)
nil

aEFEFB
Δ(aEFEFB) = (-1)a∧EFEFB + aEFB∧EF + aEFEF∧B
---
	∇(a ⊗ EFEFB) = a.BEFEF
	∇(aEFB ⊗ EF) = EFBa.EF + (-1)BaEF.EF
	∇(aEFEF ⊗ B) = (-1)EFEFa.B
a.BEFEF + EFBa.EF + (-1)BaEF.EF + (-1)EFEFa.B  =  (0)1


In [18]:
count, length = 1000, 5
count.times do |k|
  myw = word_generator(length)
  total = cobracket(myw).each_with_object(Expansion.new) do |h, expn|
    bracket(*h[:words]).each do |mono|
      mono[:coeff] *= h[:coeff]
      expn << mono
    end
  end
  result = total.show(2)
  p myw.show + " |--> " + result if result != "(0)1"
end; nil

## Compatibility condition for $\nabla$ and $\delta$

$\forall v,w \in \pi$,
$$
\delta([v,w]) = w\cdot\delta(v)- v\cdot\delta(w),
$$
where 
$$
u\cdot(x\otimes y) = [u,x]\otimes y + x\otimes[u,y].
$$
**Note**: From the last formula, we immediately have the following:
$$
u \cdot(x\wedge y) = [u,x]\wedge y - [u,y]\wedge x.
$$

In [70]:
v = word_generator(2) #samples[:s2][0] #Word.new(a[1], a[2])
w = word_generator(3) #samples[:s4][0] #Word.new(b[1])
puts (v*w).show

#--- Left-hand eq. -------------------
lhe = Expansion.new
bra = bracket(v, w)#; puts "\u{2207}(#{v}, #{w}) = " + bra.show
bra.each do |h|
  cobra = cobracket(h[:words][0])
#   puts "\t(#{h[:coeff]})\u{03B4}(#{h[:words][0].show}) = " + cobra.show(1) if h[:coeff] != 0
  cobra.each do |mono|
    mono[:coeff] *= h[:coeff]
#     puts "\t\t(#{mono[:coeff]})#{mono[:words].map(&:show).join(" \u{2227} ")}" if mono[:coeff] != 0
    lhe << mono
  end
end
printf "\nLeft-hand Eq.:  "+ lhe.show(2)

#--- Right-hand eq. -------------------
rhe = Expansion.new
2.times do |i|
  pair = [v,w].rotate(i)
  sign01 = (-1)**i
  #---
  u = pair[0]
  cobra = cobracket(pair[1]) #; printf "#{u}.\u{03B4}(#{pair[1].show}) = #{u}.(#{cobra.show(1)})\n---\n"
  cobra.each do |h|
    2.times do |j|
      couple = h[:words].rotate(j)
      x, y = couple[0], couple[1]
      sign02 = (-1)**j
      #---
      bra = bracket(u, x) #; printf "\t[#{u}, #{x}] \u{2297} #{y} \n" + "\t=(#{bra.show(1)}) \u{2297} #{y}"
      bra.each do |mono|
        term = Hash.new
        term[:coeff] = mono[:coeff]*h[:coeff]*sign01*sign02
        term[:words] = [mono[:words][0], y]
        term[:divs] = mono[:divs]
        rhe << term
      end
    end
  end
#   printf "\n---\n"
end
#-----
printf "\nRight-hand Eq.:  "+ rhe.show(2)

#-----------------------
rhe.each do |t|
  t[:coeff] *= -1
  lhe << t
end
lhe.show(2)

DF.Eca

Left-hand Eq.:  (-1)DFEa∧c + (-1)DFc∧aE + DFcE∧a + DFaE∧c + (-1)FDcE∧a + (-1)FDaE∧c
Right-hand Eq.:  DFcE∧a + (-1)FDcE∧a + (-1)DFc∧Ea + (-1)DFEa∧c + DFaE∧c + (-1)FDaE∧c

"(0)1"

# Experiments

# TODO

* [x] Expansion#show のオプションに "係数 0 の項だけ消去" を加えたい。
* [x] Compatibility が全然ダメ。
* [x] aaa などの Word について、動作が少し変。
* [x] Extension class のメソッドとして、係数 0 を消去したり、共通項で整理したりできるようにしたい。
* [ ] cobracket を再帰的に作用させて、完全に分解することに何らかの意味があるか?
* [ ] $\ell_{2}$ の計算と cobracket との関係は? とくに両者にある division について。